In [4]:
import pandas as pd
import numpy as np
import sys

import warnings
import math

df=pd.read_csv('../data/data.csv.gz', compression='gzip', sep=',')
df['timestamp'] = pd.to_datetime(df['time_stamp'], format="%Y-%m-%d %H:%M:%S").dt.tz_convert('CET')

In [5]:
stock_delta = df['stock_id'] - df['stock_id'].shift()
time_delta = (df['timestamp'] - df['timestamp'].shift()).fillna(10000).dt.total_seconds()

/home/ken/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  


In [6]:
stock_split_index=df.index[stock_delta!=0].tolist()
stock_split_index.append(len(df))

In [7]:
day_split_index=df.index[abs(time_delta)>36000].tolist()
day_split_index.insert(0, 0)
day_split_index.append(len(df))

In [8]:
stock_id_list = df['stock_id'].unique().tolist()
i = 0
for stock_index in stock_id_list:
    
    print("index: {}, id:{}".format(i, stock_index))
    i+=1

index: 0, id:101
index: 1, id:1027
index: 2, id:120
index: 3, id:139301
index: 4, id:15285
index: 5, id:160271
index: 6, id:161
index: 7, id:18634
index: 8, id:281
index: 9, id:283
index: 10, id:285
index: 11, id:300
index: 12, id:323
index: 13, id:340
index: 14, id:3524
index: 15, id:361
index: 16, id:366
index: 17, id:3966
index: 18, id:401
index: 19, id:402
index: 20, id:45
index: 21, id:46
index: 22, id:47
index: 23, id:4928
index: 24, id:5095
index: 25, id:81
index: 26, id:812
index: 27, id:819
index: 28, id:992
index: 29, id:999


In [9]:
current_stock_index = 0
time_series_list = []
time_series_all_list = []
for i in range(len(day_split_index) - 1):
    start_index = day_split_index[i]
    end_index = day_split_index[i + 1]
    day_time_series = df.iloc[start_index:end_index]
    time_series_list.append(day_time_series)
    if end_index == stock_split_index[current_stock_index + 1]:
        print("finished stock: {}, got {} time series".format(current_stock_index, len(time_series_list)))
        time_series_all_list.append(time_series_list)
        time_series_list = []
        current_stock_index += 1

finished stock: 0, got 63 time series
finished stock: 1, got 63 time series
finished stock: 2, got 63 time series
finished stock: 3, got 63 time series
finished stock: 4, got 63 time series
finished stock: 5, got 63 time series
finished stock: 6, got 63 time series
finished stock: 7, got 63 time series
finished stock: 8, got 63 time series
finished stock: 9, got 63 time series
finished stock: 10, got 63 time series
finished stock: 11, got 63 time series
finished stock: 12, got 63 time series
finished stock: 13, got 63 time series
finished stock: 14, got 63 time series
finished stock: 15, got 63 time series
finished stock: 16, got 63 time series
finished stock: 17, got 63 time series
finished stock: 18, got 63 time series
finished stock: 19, got 63 time series
finished stock: 20, got 63 time series
finished stock: 21, got 63 time series
finished stock: 22, got 63 time series
finished stock: 23, got 63 time series
finished stock: 24, got 63 time series
finished stock: 25, got 63 time ser

In [10]:
for stock_id in range(len(time_series_all_list)):
    print("handling stock_id: {}".format(stock_id))
    for day_id in range(len(time_series_all_list[stock_id])):
        # :-1 is that we don't like the last record at 17:30 which is a aggregated number.
        df = time_series_all_list[stock_id][day_id]
        # some data might miss, we must make a right join with full time series
        # and do fillna.
        df2 = df.set_index('timestamp')
        ts = df2.index.min()
        start_time_str = "{}-{:02d}-{:02d} 8:54:00".format(ts.year, ts.month, ts.day)
        start_ts = pd.Timestamp(start_time_str, tz=ts.tz)
        # periods=510 means from 9 to 17.29
        dti = pd.date_range(start_ts , periods=516, freq='min').to_series(keep_tz=True).rename('time')
        # remove from 17.25 - 17.28
        #dti.drop(dti.tail(5).head(4).index, inplace=True)
        df3 = df2.join(dti, how='right')
        if day_id == 0: # the first day, we must set the value from 8.55-8.59 as same as 9.00
            df3['last'].iloc[0] = df3['last'].iloc[5]
        else:
            df3['last'].iloc[0] = time_series_all_list[stock_id][day_id-1]['last'].iloc[-1]
        
        
        df3['last'].interpolate(method='linear', inplace=True)
        df3['volume'].iloc[:6] = df3['volume'].iloc[6] / 6
        df3['volume'].iloc[6] = df3['volume'].iloc[6] / 6
        df3['volume'].iloc[-5:] = df3['volume'].iloc[-1]/5
        
        
        df = df3.reset_index().rename({'index':'timestamp'}, axis=1)
        #df['timestamp'] = pd.to_datetime(df['time_stamp'], format="%Y-%m-%d %H:%M:%S").dt.tz_convert('Europe/Stockholm')
        df['ema_10'] = df['last'].ewm(span=10, adjust=False).mean()
        df['ema_20'] = df['last'].ewm(span=20, adjust=False).mean()
        df['diff_ema_10']=(df['ema_10'].diff()[1:]/df['ema_10']).fillna(0)
        # the first diff at 9:00 is the difference between today's open and yesterday's last.

        df['diff_ema_20']=(df['ema_20'].diff()[1:]/df['ema_20']).fillna(0)
        df['value_ema_10_beta_99'] = 0
        df['value_ema_20_beta_99'] = 0
        df['value_ema_10_beta_98'] = 0
        df['value_ema_20_beta_98'] = 0
        for iter_id in range(10):
            df['value_ema_10_beta_99'] = df['diff_ema_10'].shift(-1).fillna(0) + \
                0.99 * df['value_ema_10_beta_99'].shift(-1).fillna(0)
            df['value_ema_10_beta_98'] = df['diff_ema_10'].shift(-1).fillna(0) + \
                0.98 * df['value_ema_10_beta_98'].shift(-1).fillna(0)
            df['value_ema_20_beta_99'] = df['diff_ema_20'].shift(-1).fillna(0) + \
                0.99 * df['value_ema_20_beta_99'].shift(-1).fillna(0)
            df['value_ema_20_beta_98'] = df['diff_ema_20'].shift(-1).fillna(0) + \
                0.98 * df['value_ema_20_beta_98'].shift(-1).fillna(0)
        # drop the first row because diff is nan    
        #df.drop(0, inplace=True)
        time_series_all_list[stock_id][day_id] = df.fillna(0)

handling stock_id: 0


/home/ken/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


handling stock_id: 1
handling stock_id: 2
handling stock_id: 3
handling stock_id: 4
handling stock_id: 5
handling stock_id: 6
handling stock_id: 7
handling stock_id: 8
handling stock_id: 9
handling stock_id: 10
handling stock_id: 11
handling stock_id: 12
handling stock_id: 13
handling stock_id: 14
handling stock_id: 15
handling stock_id: 16
handling stock_id: 17
handling stock_id: 18
handling stock_id: 19
handling stock_id: 20
handling stock_id: 21
handling stock_id: 22
handling stock_id: 23
handling stock_id: 24
handling stock_id: 25
handling stock_id: 26
handling stock_id: 27
handling stock_id: 28
handling stock_id: 29


In [12]:
time_series_all_list[0][2]

,timestamp,stock_id,time_stamp,open,high,low,last,volume,time,ema_10,ema_20,diff_ema_10,diff_ema_20,value_ema_10_beta_99,value_ema_20_beta_99,value_ema_10_beta_98,value_ema_20_beta_98
0,2019-01-30 08:54:00+01:00,0.0,0,0.00,0.00,0.00,80.220,20946.0,2019-01-30 08:54:00+01:00,80.220000,80.220000,0.000000,0.000000e+00,0.004444,0.003059,0.004253,0.002913
1,2019-01-30 08:55:00+01:00,0.0,0,0.00,0.00,0.00,80.310,20946.0,2019-01-30 08:55:00+01:00,80.236364,80.228571,0.000204,1.068376e-04,0.004626,0.003286,0.004445,0.003142
2,2019-01-30 08:56:00+01:00,0.0,0,0.00,0.00,0.00,80.400,20946.0,2019-01-30 08:56:00+01:00,80.266116,80.244898,0.000371,2.034588e-04,0.004578,0.003390,0.004413,0.003250
3,2019-01-30 08:57:00+01:00,0.0,0,0.00,0.00,0.00,80.490,20946.0,2019-01-30 08:57:00+01:00,80.306822,80.268241,0.000507,2.908130e-04,0.004136,0.003271,0.004008,0.003148
4,2019-01-30 08:58:00+01:00,0.0,0,0.00,0.00,0.00,80.580,20946.0,2019-01-30 08:58:00+01:00,80.356491,80.297932,0.000618,3.697646e-04,0.003614,0.003080,0.003514,0.002971
5,2019-01-30 08:59:00+01:00,0.0,0,0.00,0.00,0.00,80.670,20946.0,2019-01-30 08:59:00+01:00,80.413492,80.333367,0.000709,4.410996e-04,0.002902,0.002758,0.002832,0.002666
6,2019-01-30 09:00:00+01:00,101.0,2019-01-30 08:00:00+00,80.50,80.76,80.30,80.760,20946.0,2019-01-30 09:00:00+01:00,80.476494,80.373999,0.000783,5.055326e-04,0.001908,0.002250,0.001879,0.002182
7,2019-01-30 09:01:00+01:00,101.0,2019-01-30 08:01:00+00,80.64,80.64,80.60,80.600,7511.0,2019-01-30 09:01:00+01:00,80.498949,80.395523,0.000279,2.677251e-04,0.001372,0.001937,0.001383,0.001893
8,2019-01-30 09:02:00+01:00,101.0,2019-01-30 08:02:00+00,80.70,80.82,80.70,80.800,9069.0,2019-01-30 09:02:00+01:00,80.553686,80.434045,0.000680,4.789219e-04,0.000517,0.001435,0.000552,0.001408
9,2019-01-30 09:03:00+01:00,101.0,2019-01-30 08:03:00+00,80.78,80.78,80.70,80.740,24929.0,2019-01-30 09:03:00+01:00,80.587561,80.463183,0.000420,3.621360e-04,-0.000010,0.001071,0.000036,0.001056


In [13]:
columns = ['timestamp','last', 'diff_ema_20', 'value_ema_20_beta_99', 'value_ema_20_beta_98', 
           'diff_ema_10', 'value_ema_10_beta_99', 'value_ema_10_beta_98', 'volume']

print(time_series_all_list[0][1][columns])

                    timestamp       last  diff_ema_20  value_ema_20_beta_99  \
0   2019-01-29 08:54:00+01:00  79.020000     0.000000              0.000853   
1   2019-01-29 08:55:00+01:00  79.033333     0.000016              0.000942   
2   2019-01-29 08:56:00+01:00  79.046667     0.000031              0.001008   
3   2019-01-29 08:57:00+01:00  79.060000     0.000044              0.001185   
4   2019-01-29 08:58:00+01:00  79.073333     0.000056              0.001420   
5   2019-01-29 08:59:00+01:00  79.086667     0.000066              0.001444   
6   2019-01-29 09:00:00+01:00  79.100000     0.000076              0.001341   
7   2019-01-29 09:01:00+01:00  79.040000    -0.000003              0.001255   
8   2019-01-29 09:02:00+01:00  79.260000     0.000262              0.000910   
9   2019-01-29 09:03:00+01:00  79.200000     0.000165              0.000779   
10  2019-01-29 09:04:00+01:00  79.240000     0.000197              0.000721   
11  2019-01-29 09:05:00+01:00  79.180000     0.00010

In [14]:
%matplotlib notebook
from ipywidgets import *

widgets.IntSlider(min=0,max=len(time_series_all_list[0]) - 1,step=1,value=10)

@interact(stock_id=widgets.IntSlider(min=0,max=len(time_series_all_list) - 1,step=1,value=10), day_id = widgets.IntSlider(min=0,max=len(time_series_all_list[0]) - 1,step=1,value=10))
def plot(stock_id, day_id ):
    df = time_series_all_list[stock_id][day_id]
    df.plot(x='timestamp', y=['last','ema_20'])
    df.plot(x='timestamp', y=['value_ema_20_beta_98'])

interactive(children=(IntSlider(value=10, description='stock_id', max=29), IntSlider(value=10, description='da…

In [16]:

from functools import reduce

# to order the columns in the order: last1,last2...last30, diff1, diff2,...diff30, value1, value2,...value30
def func(x):
    if 'timestamp' in x:
        return 0
    num = int(x.split('_')[-1])

    if 'last' in x:
        return 100 + num
    elif 'volume' in x:
        return 150 + num
    elif 'diff_ema_10' in x:
        return 200 + num
    elif 'diff_ema_20' in x:
        return 300 + num
    elif 'value_ema_10_beta_98' in x:
        return 400 + num
    elif 'value_ema_10_beta_99' in x:
        return 500 + num
    elif 'value_ema_20_beta_98' in x:
        return 600 + num
    elif 'value_ema_20_beta_99' in x:
        return 700 + num


def merge(df1, df2):
    global index
    merged = df1.set_index('timestamp').join(df2.set_index('timestamp'), how='outer')
    index += 1

    return merged.reset_index().rename({'index':'timestamp'}, axis=1)


columns_wanted = ['timestamp','last', 'diff_ema_20', 'value_ema_20_beta_99', 'value_ema_20_beta_98', 
           'diff_ema_10', 'value_ema_10_beta_99', 'value_ema_10_beta_98', 'volume']
df_day_list = []

for day_id in range(len(time_series_all_list[0])):
    print("processing day: {}".format(day_id))
    df_list = []
    for i in range(len(stock_id_list)):
        rename_map = {'last': 'last_' + str(i),
                      'diff_ema_20':'diff_ema_20_' + str(i),
                      'value_ema_20_beta_99':'value_ema_20_beta_99_' + str(i),
                      'value_ema_20_beta_98':'value_ema_20_beta_98_' + str(i),
                      'diff_ema_10':'diff_ema_10_' + str(i),
                      'value_ema_10_beta_99':'value_ema_10_beta_99_' + str(i),
                      'value_ema_10_beta_98':'value_ema_10_beta_98_'+ str(i),
                      'volume': 'volume_' + str(i)
                     }
        df = time_series_all_list[i][day_id][columns_wanted].rename(rename_map, axis=1)
        df_list.append(df)

    index = 0
    df_merged_daily = reduce(merge, df_list)
    columns = df_merged_daily.columns.tolist()
    columns.sort(key=func)
    df_merged_sorted = df_merged_daily[columns]
    
    df_day_list.append(df_merged_sorted)
    if day_id == 0:
        df_merged = df_merged_sorted
    else:
        df_merged = df_merged.append(df_merged_sorted)

processing day: 0
processing day: 1
processing day: 2
processing day: 3
processing day: 4
processing day: 5
processing day: 6
processing day: 7
processing day: 8
processing day: 9
processing day: 10
processing day: 11
processing day: 12
processing day: 13
processing day: 14
processing day: 15
processing day: 16
processing day: 17
processing day: 18
processing day: 19
processing day: 20
processing day: 21
processing day: 22
processing day: 23
processing day: 24
processing day: 25
processing day: 26
processing day: 27
processing day: 28
processing day: 29
processing day: 30
processing day: 31
processing day: 32
processing day: 33
processing day: 34
processing day: 35
processing day: 36
processing day: 37
processing day: 38
processing day: 39
processing day: 40
processing day: 41
processing day: 42
processing day: 43
processing day: 44
processing day: 45
processing day: 46
processing day: 47
processing day: 48
processing day: 49
processing day: 50
processing day: 51
processing day: 52
pro

In [17]:

df_merged

,timestamp,last_0,last_1,last_2,last_3,last_4,last_5,last_6,last_7,last_8,...,value_ema_20_beta_99_20,value_ema_20_beta_99_21,value_ema_20_beta_99_22,value_ema_20_beta_99_23,value_ema_20_beta_99_24,value_ema_20_beta_99_25,value_ema_20_beta_99_26,value_ema_20_beta_99_27,value_ema_20_beta_99_28,value_ema_20_beta_99_29
0,2019-01-28 08:54:00+01:00,0.000,0.000,0.00,0.00,0.00,0.000,0.00,0.000,0.000,...,-0.000894,-0.000687,0.000302,-0.000608,0.000325,0.000070,-0.000328,-1.312065e-03,0.000389,0.000239
1,2019-01-28 08:55:00+01:00,0.000,0.000,0.00,0.00,0.00,0.000,0.00,0.000,0.000,...,-0.000877,-0.000609,0.000539,-0.000770,0.000473,0.000148,-0.000229,-1.397129e-03,0.000460,0.000335
2,2019-01-28 08:56:00+01:00,0.000,0.000,0.00,0.00,0.00,0.000,0.00,0.000,0.000,...,-0.000671,-0.000351,0.000924,-0.000828,0.000543,0.000219,-0.000148,-1.456723e-03,0.000616,0.000499
3,2019-01-28 08:57:00+01:00,0.000,0.000,0.00,0.00,0.00,0.000,0.00,0.000,0.000,...,-0.000541,-0.000075,0.001252,-0.000881,0.000585,0.000367,-0.000034,-1.493094e-03,0.000785,0.000650
4,2019-01-28 08:58:00+01:00,0.000,0.000,0.00,0.00,0.00,0.000,0.00,0.000,0.000,...,-0.000556,-0.000133,0.001529,-0.000962,0.000537,0.000489,0.000092,-1.605756e-03,0.000849,0.000904
5,2019-01-28 08:59:00+01:00,0.000,0.000,0.00,0.00,0.00,0.000,0.00,0.000,0.000,...,-0.000570,-0.000104,0.001784,-0.000822,0.000624,0.000587,0.000227,-1.788277e-03,0.001038,0.001175
6,2019-01-28 09:00:00+01:00,80.780,115.200,210.80,223.70,219.55,83.000,397.10,200.300,92.500,...,-0.000584,-0.000057,0.002018,-0.000726,0.000727,0.000663,0.000372,-2.054369e-03,0.001238,0.001309
7,2019-01-28 09:01:00+01:00,80.640,115.600,210.80,223.70,219.10,83.020,395.70,199.600,92.540,...,-0.000268,0.000166,0.002216,-0.000640,0.000899,0.000733,0.000812,-2.084324e-03,0.001390,0.001595
8,2019-01-28 09:02:00+01:00,80.620,115.800,211.10,223.00,217.80,83.110,395.20,199.600,92.500,...,0.000203,0.000680,0.002317,-0.000197,0.000841,0.000656,0.001135,-1.630403e-03,0.001780,0.001772
9,2019-01-28 09:03:00+01:00,80.580,115.950,210.70,223.00,218.40,83.070,395.40,199.450,92.340,...,0.000419,0.000959,0.002305,0.000110,0.000620,0.000933,0.001323,-1.322631e-03,0.002022,0.001808


In [18]:
columns = df_merged_sorted.columns
last_columns = []
volume_columns = []
diff_ema_10_columns = []
diff_ema_20_columns = []
value_ema_10_beta_98_columns = []
value_ema_10_beta_99_columns = []
value_ema_20_beta_98_columns = []
value_ema_20_beta_99_columns = []
for item in columns:
    if 'last' in item:
        last_columns.append(item)
    elif 'volume' in item:
        volume_columns.append(item)
    elif 'diff_ema_10' in item:
        diff_ema_10_columns.append(item)
    elif 'diff_ema_20' in item:
        diff_ema_20_columns.append(item)
    elif 'value_ema_10_beta_98' in item:
        value_ema_10_beta_98_columns.append(item)
    elif 'value_ema_10_beta_99' in item:
        value_ema_10_beta_99_columns.append(item)
    elif 'value_ema_20_beta_98' in item:
        value_ema_20_beta_98_columns.append(item)
    elif 'value_ema_20_beta_99' in item:
        value_ema_20_beta_99_columns.append(item)



In [19]:
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_10_columns + value_ema_10_beta_98_columns].to_csv('data-prep-ema10-beta98.csv')
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_10_columns + value_ema_10_beta_99_columns].to_csv('data-prep-ema10-beta99.csv')
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_20_columns + value_ema_20_beta_98_columns].to_csv('data-prep-ema20-beta98.csv')
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_20_columns + value_ema_20_beta_99_columns].to_csv('data-prep-ema20-beta99.csv')